# Hungary Lap Times Data Cleaning Notebook
**Author**: Yuchun Wang  
**Objective**: Clean the lap times data of the Hungarian Grand Prix in strict adherence to predefined guidelines.

## Data Cleaning Guidelines
1. For each driver, retain only their **fastest lap time** record.  
2. Standardize time format to retain only the `minutes:seconds.milliseconds` part (drop hour components and "0 days " prefix).  
3. Directly delete any row containing blank values in `Sector 1/2/3` or `Lap Time` during cleaning.  
4. Remove the `Position` column (irrelevant to lap time analysis).  
5. Remove the `Time` column (cumulative race time, unrelated to the core objective of fastest lap analysis).

In [2]:
import numpy as np
import pandas as pd

***Load and Inspect Raw Data***

In [3]:
# Load data 
df = pd.read_csv("lap_times_2025_round_14.csv") 

# Check first 5 rows to verify column names and data format
print("First 5 rows of raw data:")
display(df.head())

# List all column names to confirm required fields exist
print("\nAll column names:")
print(df.columns.tolist())

First 5 rows of raw data:


,Driver,Lap Number,Lap Time,Position,Time,Sector 1,Sector 2,Sector 3
0,NOR,1.0,0 days 00:01:30.120000,5.0,0 days 00:59:18.340000,NaN,0 days 00:00:31.396000,0 days 00:00:23.329000
1,NOR,2.0,0 days 00:01:23.964000,5.0,0 days 01:00:42.304000,0 days 00:00:30.209000,0 days 00:00:30.346000,0 days 00:00:23.409000
2,NOR,3.0,0 days 00:01:23.223000,4.0,0 days 01:02:05.527000,0 days 00:00:29.792000,0 days 00:00:30.097000,0 days 00:00:23.334000
3,NOR,4.0,0 days 00:01:22.877000,4.0,0 days 01:03:28.404000,0 days 00:00:30.125000,0 days 00:00:29.665000,0 days 00:00:23.087000
4,NOR,5.0,0 days 00:01:22.639000,4.0,0 days 01:04:51.043000,0 days 00:00:29.930000,0 days 00:00:29.672000,0 days 00:00:23.037000



All column names:
['Driver', 'Lap Number', 'Lap Time', 'Position', 'Time', 'Sector 1', 'Sector 2', 'Sector 3']


***Remove 'Position' Column***

In [ ]:
# Check if 'Position' column exists and remove it
if "Position" in df.columns:
    df = df.drop(columns=["Position"])
    print("'Position' column has been removed.")
else:
    print("No 'Position' column found in the data.")

# Confirm columns after removal
print("\nColumn names after removal:")
print(df.columns.tolist())

'Position' column has been removed.

Column names after removal:
['Driver', 'Lap Number', 'Lap Time', 'Time', 'Sector 1', 'Sector 2', 'Sector 3']


***Drop Rows with Blank Sector or Lap Time***

In [5]:
# Define columns to check for blanks (adjust column names if different)
columns_to_check = ["Lap Time", "Sector 1", "Sector 2", "Sector 3"]

# Count missing values before processing
print("Missing values before cleaning:")
print(df[columns_to_check].isnull().sum())

# Drop rows with any missing value in the specified columns
df_clean = df.dropna(subset=columns_to_check, how="any")

# Confirm row count change
print(f"\nAfter dropping blank rows, data reduced from {len(df)} to {len(df_clean)} rows.")

Missing values before cleaning:
Lap Time     0
Sector 1    20
Sector 2     0
Sector 3     0
dtype: int64

After dropping blank rows, data reduced from 1368 to 1348 rows.


***Clean Time Format (Retain Only Minutes-Onward)***

In [ ]:
def clean_time_to_minutes_onward(time_str):
    """
    Clean time string: 
    1. Remove "0 days " prefix 
    2. Retain only minutes:seconds.milliseconds (drop hour component)
    """
    time_str = str(time_str).strip()  # Ensure string type and remove extra spaces
    
    # Step 1: Remove "0 days " prefix if present
    if "days" in time_str:
        time_str = time_str.split("days ")[1].strip()
    
    # Step 2: Extract minutes-onward (handle both "hh:mm:ss.ms" and "mm:ss.ms" formats)
    parts = time_str.split(":")
    if len(parts) == 3:  # Format: hh:mm:ss.ms → keep mm:ss.ms
        cleaned_str = f"{parts[1]}:{parts[2]}"
    elif len(parts) == 2:  # Format: mm:ss.ms → already correct, keep as-is
        cleaned_str = time_str
    else:  # Unexpected format (e.g., only seconds) → keep original to avoid data loss
        cleaned_str = time_str
    
    return cleaned_str


# Apply cleaning to Lap Time and all Sector columns
time_columns = ["Lap Time", "Sector 1", "Sector 2", "Sector 3"]
for col in time_columns:
    df_clean[col] = df_clean[col].apply(clean_time_to_minutes_onward)

# Verify cleaned time format (should show "mm:ss.ms")
print("Cleaned time format (minutes:seconds.milliseconds) - First 5 rows:")
display(df_clean[time_columns].head())

Cleaned time format (minutes:seconds.milliseconds) - First 5 rows:


C:\Users\lenovo\AppData\Local\Temp\ipykernel_44552\2983580594.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[col] = df_clean[col].apply(clean_time_to_minutes_onward)


,Lap Time,Sector 1,Sector 2,Sector 3
1,01:23.964000,00:30.209000,00:30.346000,00:23.409000
2,01:23.223000,00:29.792000,00:30.097000,00:23.334000
3,01:22.877000,00:30.125000,00:29.665000,00:23.087000
4,01:22.639000,00:29.930000,00:29.672000,00:23.037000
5,01:22.181000,00:29.880000,00:29.313000,00:22.988000


***Retain the Fastest Lap Time per Driver***

Temporarily convert the cleaned "mm:ss.ms" string to seconds for accurate fastest lap comparison (then drop the temporary column to keep original time format):

In [12]:
def minutes_onward_to_seconds(time_str):
    """
    Temporarily convert time strings in "mm:ss.ms" or "mm:ss" format to total seconds (compatible with cases without milliseconds).

    """
    parts = time_str.split(":")
    if len(parts) == 2: 
        mm, ss_ms = parts
        # Handle splitting of seconds and milliseconds (compatible with cases without milliseconds)
        if "." in ss_ms: 
            ss, ms = ss_ms.split(".")
        else: 
            ss = ss_ms
            ms = "0"
        # Convert to total seconds 
        total_seconds = int(mm) * 60 + int(ss) + int(ms.ljust(3, "0")[:3]) / 1000
        return total_seconds
    else:
        # Unexpected format 
        return float("inf")

In [13]:
# Apply the optimized function to add a temporary column for sorting (will be removed later)
df_clean["temp_lap_time_seconds"] = df_clean["Lap Time"].apply(minutes_onward_to_seconds)

# Check conversion results of the temporary column (ensure no anomalies)
print("Temporary seconds conversion results (first 5 rows):")
display(df_clean[["Lap Time", "temp_lap_time_seconds"]].head())

# Group by driver and get indices of rows with the minimum temporary seconds (fastest laps)
fastest_lap_indices = df_clean.groupby("Driver")["temp_lap_time_seconds"].idxmin()

# Filter fastest lap records and drop the temporary column (preserve original cleaned time strings)
df_fastest = df_clean.loc[fastest_lap_indices].drop(columns=["temp_lap_time_seconds"]).reset_index(drop=True)

# Verify results: 1 fastest lap record per driver
print(f"\nCleaned data contains {len(df_fastest)} drivers' fastest laps")
print("First 5 rows of fastest lap data (minutes:seconds.ms format):")
display(df_fastest[["Driver", "Lap Time", "Sector 1", "Sector 2", "Sector 3"]].head())

Temporary seconds conversion results (first 5 rows):


C:\Users\lenovo\AppData\Local\Temp\ipykernel_44552\3896292562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["temp_lap_time_seconds"] = df_clean["Lap Time"].apply(minutes_onward_to_seconds)


,Lap Time,temp_lap_time_seconds
1,01:23.964000,83.964
2,01:23.223000,83.223
3,01:22.877000,82.877
4,01:22.639000,82.639
5,01:22.181000,82.181



Cleaned data contains 20 drivers' fastest laps
First 5 rows of fastest lap data (minutes:seconds.ms format):


,Driver,Lap Time,Sector 1,Sector 2,Sector 3
0,ALB,01:20.779000,00:28.863000,00:29.021000,00:22.895000
1,ALO,01:20.113000,00:29.232000,00:28.298000,00:22.583000
2,ANT,01:20.745000,00:29.216000,00:28.731000,00:22.798000
3,BEA,01:21.989000,00:29.662000,00:29.236000,00:23.091000
4,BOR,01:20.705000,00:29.184000,00:28.684000,00:22.837000


***Finalize and Save Data***

In [ ]:
# Define final columns to retain 
final_columns = [
    "Driver",        
    "Lap Time",      
    "Sector 1",      
    "Sector 2",      
    "Sector 3"      
]
df_final = df_fastest[final_columns]

# Save to CSV 
output_file = "hungary_fastest_laps.csv"
df_final.to_csv(output_file, index=False)

# Display sample of the final data
print(f"\nFinal data saved to: {output_file}")
print("Sample of final data:")
display(df_final.head())


Final data saved to: hungary_fastest_laps.csv
Sample of final data:


,Driver,Lap Time,Sector 1,Sector 2,Sector 3
0,ALB,01:20.779000,00:28.863000,00:29.021000,00:22.895000
1,ALO,01:20.113000,00:29.232000,00:28.298000,00:22.583000
2,ANT,01:20.745000,00:29.216000,00:28.731000,00:22.798000
3,BEA,01:21.989000,00:29.662000,00:29.236000,00:23.091000
4,BOR,01:20.705000,00:29.184000,00:28.684000,00:22.837000


***Generate Cleaning Logic***

In [16]:
# Calculate data volume changes across stages
original_rows = len(df)
after_drop_position = len(df)  # Row count unchanged (only column removed)
after_drop_blanks = len(df_clean)
dropped_blank_rows = original_rows - after_drop_blanks
final_drivers = len(df_fastest)

# Print the log
print("\n===== Comprehensive Data Cleaning Log =====")
print(f"1. Original row count: {original_rows}")
print(f"2. Row count after removing 'Position' column: {after_drop_position} (only column removed; rows unchanged)")
print(f"3. Row count after dropping blank/invalid rows: {after_drop_blanks} (total {dropped_blank_rows} rows removed)")
print(f"4. Number of drivers retained: {final_drivers} (1 fastest lap per driver)")
print(f"5. Time format handling: Standardized to 'mm:ss.ms' or 'mm:ss' (hour component and '0 days' prefix removed)")
print(f"6. Final output file path: {output_file}")
print(f"7. Final data columns: {final_columns}")


===== Comprehensive Data Cleaning Log =====
1. Original row count: 1368
2. Row count after removing 'Position' column: 1368 (only column removed; rows unchanged)
3. Row count after dropping blank/invalid rows: 1348 (total 20 rows removed)
4. Number of drivers retained: 20 (1 fastest lap per driver)
5. Time format handling: Standardized to 'mm:ss.ms' or 'mm:ss' (hour component and '0 days' prefix removed)
6. Final output file path: hungary_fastest_laps.csv
7. Final data columns: ['Driver', 'Lap Time', 'Sector 1', 'Sector 2', 'Sector 3']


### Subsequent Work Plan 
1. **Data Integration**: Merge cleaned data of three circuits and supplement circuit characteristic fields;
2. **Feature Engineering**: Extract lap time indicators (average/distribution) and quantify circuit characteristics;
3. **Modeling & Analysis**: Build regression models to quantify the impact of circuit characteristics on lap times;
4. **Outcome Delivery**: Generate visual charts and analysis reports.